In [40]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import copy
import json

In [41]:
PIECES_TO_NUM = {
    "square": 0,
    "white_pawn": 1,
    "white_rook": 2,
    "white_knight": 3,
    "white_bishop": 4,
    "white_king": 5,
    "white_queen": 6,
    "white_unknown": 7,
    "black_pawn": -1,
    "black_rook": -2,
    "black_knight": -3,
    "black_bishop": -4,
    "black_king": -5,
    "black_queen": -6,
    "black_unknown": -7,
}

NUM_TO_PIECE = {
    0: "square",
    1: "white_pawn",
    2: "white_rook",
    3: "white_knight",
    4: "white_bishop",
    5: "white_king",
    6: "white_queen",
    7: "white_unknown",
    -1: "black_pawn",
    -2: "black_rook",
    -3: "black_knight",
    -4: "black_bishop",
    -5: "black_king",
    -6: "black_queen",
    -7: "black_unknown"
}

In [42]:
piece_images = {
    piece: Image.open(f'assets/{piece}.png') for piece in PIECES_TO_NUM.keys() if piece != "square"
}

In [43]:
game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  -7],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  7],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

def draw_chessboard(board, save=False):
    board = np.flipud(board)
    fig, ax = plt.subplots()

    chessboard_pattern = np.zeros((8, 8))
    chessboard_pattern[1::2, ::2] = 1
    chessboard_pattern[::2, 1::2] = 1

    ax.imshow(chessboard_pattern, cmap='gray', interpolation='none')

    for i in range(8):
        for j in range(8):
            piece_value = board[i, j]
            if piece_value != 0:
                piece_name = NUM_TO_PIECE[piece_value]
                piece_image = piece_images[piece_name]
                ax.imshow(piece_image, extent=[j - 0.5, j + 0.5, i - 0.5, i + 0.5])

    ax.set_xticks(np.arange(8))
    ax.set_yticks(np.arange(8))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks(np.arange(-0.5, 8, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 8, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)

    if save:
        fig.savefig(f"{save}.png")

    #plt.show()
    matplotlib.pyplot.close()
    
draw_chessboard(game_state)

In [44]:
import numpy as np
import copy

def modify_game_state(game_state, move):
    new_game_state = copy.copy(np.flipud(game_state))  # A1 is 0,0

    if " -> " in move or " x " in move or " + " in move:
        if " -> " in move:
            init_pos, new_pos = move.split(" -> ")
        elif " x " in move:
            init_pos, new_pos = move.split(" x ")
        else:  # " + " case, handled similarly
            init_pos, new_pos = move.split(" + ")

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)
        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif "O-O-O" in move:  # Queenside castling
        if "w" in move:  # Assuming the color is specified in the move string
            new_game_state[0, 2] = new_game_state[0, 4]
            new_game_state[0, 3] = new_game_state[0, 0]
            new_game_state[0, 0] = 0
            new_game_state[0, 4] = 0
        elif "b" in move:
            new_game_state[7, 2] = new_game_state[7, 4]
            new_game_state[7, 3] = new_game_state[7, 0]
            new_game_state[7, 0] = 0
            new_game_state[7, 4] = 0

    elif "O-O" in move:  # Kingside castling
        if "w" in move:
            new_game_state[0, 6] = new_game_state[0, 4]
            new_game_state[0, 5] = new_game_state[0, 7]
            new_game_state[0, 7] = 0
            new_game_state[0, 4] = 0
        elif "b" in move:
            new_game_state[7, 6] = new_game_state[7, 4]
            new_game_state[7, 5] = new_game_state[7, 7]
            new_game_state[7, 7] = 0
            new_game_state[7, 4] = 0

    return np.flipud(new_game_state)


In [45]:

game_fix = [
    "Ng1 -> Nf3",
    "Ng8 -> Nf6",
    "g2 -> g3",
    "d7 -> d5",
    "Bf1 -> Bg2",
    "c7 -> c5",
    "wO-O",  
    "Nb8 -> Nc6",
    "d2 -> d4",
    "e7 -> e6",
    "c2 -> c4",
    "d5 x c4",
    "Nf3 -> Ne5",
    "Bc8 -> Bd7",
    "Nb1 -> Na3",
    "c5 x d4",
    "Na3 x Nc4",
    "Bf8 -> Bc5",
    "Qd1 -> Qb3",
    "bO-O",  
    "Bc1 -> Bf4",
    "Qd8 -> Qc8",
    "Ne5 -> Nd3",
    "Bc5 -> Be7",
    "Rf1 -> Rc1",
    "Nf6 -> Nd5",
    "Bg2 x Bd5",
    "e6 x d5",
    "Nc4 -> Ne5",
    "Rf8 -> Rd8",
    "Qb3 x Qd5",
    "Bd7 -> Be8",
    "Qd5 -> Qb3",
    "Kg8 -> Kf8",
    "Ne5 x Nc6",
    "b7 x c6",
    "Qb3 -> Qc4",
    "Qc8 -> Qf5",
    "Bf4 -> Bd2",
    "Ra8 -> Rb8",
    "Bd2 -> Bb4",
    "Be7 x Bb4",
    "Nd3 x Nb4",
    "c6 -> c5",
    "Nb4 -> Nd3",
    "Be8 -> Bb5",
    "Qc4 + Qc5",
    "Qf5 x Qc5",
    "Rc1 x Rc5",
    "Bb5 x Bd3",
    "e2 x d3",
    "Rb8 x Rb2",
    "Rc5 -> Rc7",
    "Rb2 -> Rd2",
    "Ra1 -> Rb1",
    "Rd2 x Rd3",
    "Kg1 -> Kg2",
    "Rd3 -> Rc3",
    "Rc7 x Ra7",
    "Rc3 -> Rc6",
]



game_moving = [
    "d2 -> d4",
    "Ng8 -> Nf6",
    "c2 -> c4",
    "e7 -> e6",
    "g2 -> g3",
    "Bf8 + Bb4",
    "Bc1 -> Bd2",
    "Bb4 -> Be7",
    "Bf1 -> Bg2",
    "d7 -> d5",
    "Ng1 -> Nf3",
    "bO-O",
    "wO-O",
    "Nf6 -> Ne4",
    "Bd2 -> Bf4",
    "f7 -> f5",
    "Nf3 -> Ne5",
    "Nb8 -> Nd7",
    "Nb1 -> Nc3",
    "c7 -> c6",
    "Nc3 x Ne4",
    "f5 x e4",
    "f2 -> f3",
    "e4 x f3",
    "Ne5 x Nf3",
    "c6 -> c5",
    "d4 x c5",
    "Be7 + Bc5",
    "Kg1 -> Kh1",
    "Nd7 -> Nf6",
    "Nf3 -> Ne5",
    "Bc5 -> Bd6",
    "Qd1 -> Qd4",
    "Qd8 -> Qb6",
    "Qd4 x Qb6",
    "a7 x b6",
    "Ne5 -> Nd3",
    "Rf8 -> Rd8",
    "Bf4 x Bd6",
    "Rd8 x Rd6",
    "Nd3 -> Ne5",
    "Bc8 -> Bd7",
    "Rf1 -> Rd1",
    "Bd7 -> Bc6",
    "e2 -> e4",
    "Rd6 -> Rd8",
    "e4 x d5",
    "e6 x d5",
    "Ne5 x Nc6",
    "b7 x c6",
    "c4 x d5",
    "Nf6 x Nd5",
    "a2 -> a3",
    "Rd8 -> Rd6",
    "Rd1 -> Rd2",
    "Ra8 -> Rd8",
    "Ra1 -> Rd1",
    "Kg8 -> Kf7",
    "Kh1 -> Kg1",
    "Kf7 -> Ke6",
    "Bg2 + Bh3",
    "Ke6 -> Ke7",
    "Rd1 + Re1",
]



frames_fix = {
    1: 60,
    2: 130,
    3: 240,
    4: 300,
    5: 380,
    6: 470,
    7: 611,
    8: 670,
    9: 828,
    10: 890,
    11: 1015,
    12: 1146,
    13: 1277,
    14: 1355,
    15: 1515,
    16: 1695,
    17: 1890,
    18: 2070,
    19: 2173,
    20: 2402,
    21: 2570,
    22: 2607,
    23: 2758,
    24: 2901,
    25: 3036,
    26: 3149,
    27: 3329,
    28: 3478,
    29: 3640,
    30: 3716,
    31: 3889,
    32: 4029,
    33: 4159,
    34: 4219,
    35: 4450,
    36: 4581,
    37: 4654,
    38: 4823,
    39: 4998,
    40: 5063,
    41: 5298,
    42: 5499,
    43: 5703,
    44: 5763,
    45: 5834,
    46: 6001,
    47: 6160,
    48: 6350,
    49: 6452,
    50: 6630,
    51: 6755,
    52: 6965,
    53: 7042,
    54: 7140,
    55: 7256,
    56: 7367,
    57: 7505,
    58: 7586,
    59: 7807,
    60: 7900
}

frames_moving = {
    1: 61,
    2: 122,
    3: 256,
    4: 328,
    5: 436,
    6: 521,
    7: 643,
    8: 754,
    9: 856,
    10: 935,
    11: 1094,
    12: 1245,
    13: 1370,
    14: 1470,
    15: 1620,
    16: 1696,
    17: 1794,
    18: 1912,
    19: 2018,
    20: 2118,
    21: 2310,
    22: 2470,
    23: 2546,
    24: 2770,
    25: 2886,
    26: 2946,
    27: 3077,
    28: 3274,
    29: 3324,
    30: 3503,
    31: 3631,
    32: 3742,
    33: 3867,
    34: 3980,
    35: 4115,
    36: 4287,
    37: 4437,
    38: 4520,
    39: 4672,
    40: 4791,
    41: 4881,
    42: 4984,
    43: 5094,
    44: 5162,
    45: 5279,
    46: 5399,
    47: 5560,
    48: 5741,
    49: 5900,
    50: 6043,
    51: 6232,
    52: 6336,
    53: 6440,
    54: 6504,
    55: 6670,
    56: 6732,
    57: 6890,
    58: 6944,
    59: 7002,
    60: 7156,
    61: 7250,
    62: 7339,
    63: 7471
}


In [46]:
data = {} # data contains the list of the move and the game_state + frame_number after each move
data["moves"] = game_fix
data["game_states"] = []

In [47]:
initial_game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

# We always 
data["game_states"].append({"frame": 0,
                            "gs": initial_game_state.tolist()
                           })

draw_chessboard(initial_game_state, "step1/games/fix/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])

# In you case, the frame number won't be 1, this is just an example here.
data["game_states"].append({"frame": frames_fix.get(1),
                            "gs": new_gs.tolist()
                           })
draw_chessboard(new_gs, "step1/games/fix/gs_1")

i = 2
for j, g in enumerate(game_fix[1:]):
    new_gs = modify_game_state(new_gs, g)
    data["game_states"].append({"frame": frames_fix.get(i),
                            "gs": new_gs.tolist()
                           })
    draw_chessboard(new_gs, f"step1/games/fix/gs_{i}")
    i = i + 1

In [48]:
# save the data in a json file
json.dump(data, open("step1/games/game_fix.json", "w"))

In [49]:
initial_game_state[0, :] = -7
initial_game_state[1, :] = -7

initial_game_state[6, :] = 7
initial_game_state[7, :] = 7

draw_chessboard(initial_game_state, "step1/games/fix/unknown/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])
draw_chessboard(new_gs, "step1/games/fix/unknown/gs_1")

i = 2
for j, g in enumerate(game_fix[1:]):
    new_gs = modify_game_state(new_gs, g)
    draw_chessboard(new_gs, f"step1/games/fix/unknown//gs_{i}")
    i = i + 1

In [50]:
# You can retrieve the game state from the json file
# but don't forget to convert the list to a numpy array
draw_chessboard(np.array(data["game_states"][0]["gs"]))

In [51]:
data = {} # data contains the list of the move and the game_state + frame_number after each move
data["moves"] = game_moving
data["game_states"] = []

In [52]:
initial_game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

# We always 
data["game_states"].append({"frame": 0,
                            "gs": initial_game_state.tolist()
                           })

draw_chessboard(initial_game_state, "step1/games/moving/gs_0")

new_gs = modify_game_state(initial_game_state, game_moving[0])

# In you case, the frame number won't be 1, this is just an example here.
data["game_states"].append({"frame": frames_moving.get(1),
                            "gs": new_gs.tolist()
                           })
draw_chessboard(new_gs, "step1/games/moving/gs_1")

i = 2
for j, g in enumerate(game_moving[1:]):
    new_gs = modify_game_state(new_gs, g)
    data["game_states"].append({"frame": frames_moving.get(i),
                            "gs": new_gs.tolist()
                           })
    draw_chessboard(new_gs, f"step1/games/moving/gs_{i}")
    i = i + 1

In [53]:
# save the data in a json file
json.dump(data, open("step1/games/game_moving.json", "w"))

In [54]:
initial_game_state[0, :] = -7
initial_game_state[1, :] = -7

initial_game_state[6, :] = 7
initial_game_state[7, :] = 7

draw_chessboard(initial_game_state, "step1/games/moving/unknown/gs_0")

new_gs = modify_game_state(initial_game_state, game_moving[0])
draw_chessboard(new_gs, "step1/games/moving/unknown/gs_1")

i = 2
for j,g in enumerate(game_moving[1:]):
    new_gs = modify_game_state(new_gs, g)
    draw_chessboard(new_gs, f"step1/games/moving/unknown//gs_{i}")
    i = i + 1

In [55]:
# You can retrieve the game state from the json file
# but don't forget to convert the list to a numpy array
draw_chessboard(np.array(data["game_states"][0]["gs"]))